In [1]:
#поработаем с данными о транзакциях в ритейле
import pandas as pd

retail = pd.read_csv('/home/jupyter-a.kosheleva-14/data.csv', encoding='ISO-8859-1')

In [2]:
retail

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,12/1/2010 8:26,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,12/1/2010 8:26,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,12/1/2010 8:26,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,12/1/2010 8:26,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,12/1/2010 8:26,3.39,17850.0,United Kingdom
...,...,...,...,...,...,...,...,...
541904,581587,22613,PACK OF 20 SPACEBOY NAPKINS,12,12/9/2011 12:50,0.85,12680.0,France
541905,581587,22899,CHILDREN'S APRON DOLLY GIRL,6,12/9/2011 12:50,2.10,12680.0,France
541906,581587,23254,CHILDRENS CUTLERY DOLLY GIRL,4,12/9/2011 12:50,4.15,12680.0,France
541907,581587,23255,CHILDRENS CUTLERY CIRCUS PARADE,4,12/9/2011 12:50,4.15,12680.0,France


In [3]:
retail_columns = retail.columns.tolist()

In [4]:
retail.shape[0]

541909

In [5]:
#сколько дубликатов в датасете
retail.duplicated().sum()

retail_no_dupl = retail.drop_duplicates()

retail.shape[0] - retail_no_dupl.shape[0]

5268

In [8]:
#сколько отмененных заказов
retail.InvoiceNo.str.startswith('C').sum()

9288

In [9]:
#оставим только заказы, где количество > 0
more_than_0 = retail[retail.Quantity > 0]
more_than_0.shape

(531285, 8)

In [12]:
#клиенты из Германии
german_customers = retail[retail.Country == 'Germany']

In [13]:
german_customers

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
1109,536527,22809,SET OF 6 T-LIGHTS SANTA,6,12/1/2010 13:04,2.95,12662.0,Germany
1110,536527,84347,ROTATING SILVER ANGELS T-LIGHT HLDR,6,12/1/2010 13:04,2.55,12662.0,Germany
1111,536527,84945,MULTI COLOUR SILVER T-LIGHT HOLDER,12,12/1/2010 13:04,0.85,12662.0,Germany
1112,536527,22242,5 HOOK HANGER MAGIC TOADSTOOL,12,12/1/2010 13:04,1.65,12662.0,Germany
1113,536527,22244,3 HOOK HANGER MAGIC GARDEN,12,12/1/2010 13:04,1.95,12662.0,Germany
...,...,...,...,...,...,...,...,...
541801,581578,22993,SET OF 4 PANTRY JELLY MOULDS,12,12/9/2011 12:16,1.25,12713.0,Germany
541802,581578,22907,PACK OF 20 NAPKINS PANTRY DESIGN,12,12/9/2011 12:16,0.85,12713.0,Germany
541803,581578,22908,PACK OF 20 NAPKINS RED APPLES,12,12/9/2011 12:16,0.85,12713.0,Germany
541804,581578,23215,JINGLE BELL HEART ANTIQUE SILVER,12,12/9/2011 12:16,2.08,12713.0,Germany


In [14]:
#оставим тех, кто совершил более N транзакций, где N - 80й процентиль
trans_count = german_customers.groupby(['CustomerID'], as_index=False) \
    .agg({'InvoiceNo': 'nunique'})

quantile_german = trans_count.InvoiceNo.quantile(0.8)

quantile_german

germany_top = trans_count.query('InvoiceNo > @quantile_german').CustomerID


top_retail_germany = retail.query('CustomerID in @germany_top')

top_retail_germany

In [20]:
# какой продукт добавлялся в корзину чаще всего?
top_retail_germany.groupby(['StockCode']) \
    .agg({'Quantity':'count'}) \
    .sort_values('Quantity', ascending = False)

,Quantity
StockCode,
POST,219
22326,63
22423,55
22328,45
22467,37
...,...
23008,1
22999,1
20717,1


In [22]:
#сколько заработано на каждом продукте
retail['Revenue'] = retail.Quantity*retail.UnitPrice

In [23]:
#для каждой транзакции посчитаем сумму заказа и отберем топ-5
retail.groupby('InvoiceNo') \
    .agg({'Revenue':'sum'}) \
    .sort_values('Revenue', ascending=False) \
    .head(5) 

,Revenue
InvoiceNo,
581483,168469.60
541431,77183.60
574941,52940.94
576365,50653.91
556444,38970.00


In [ ]:
#теперь поработаем с датасетами продажи продуктов через телемаркетинг, сначала считаем файлы

In [163]:
sales1 = pd.read_csv('shared/homeworks/python_ds_miniprojects/5_subsid/subsid/tm_sales_1.csv', sep = ';')

In [164]:
sales2 = pd.read_csv('shared/homeworks/python_ds_miniprojects/5_subsid/subsid/tm_sales_2.csv',sep = ';')

In [165]:
sales3 = pd.read_csv('shared/homeworks/python_ds_miniprojects/5_subsid/subsid/tm_sales_3.csv', sep = ';')

In [166]:
logs = pd.read_csv('shared/homeworks/python_ds_miniprojects/5_subsid/subsid/prod_activations_logs.csv',sep = ';')

In [167]:
logs

,SUBS_ID,PROD_ID,START_DTTM,END_DTTM
1,id4651830,1954,20-03-2020 14:59,01-12-2020 00:00
5,id7646509,6431,19-03-2020 13:00,19-03-2020 13:03
2,id7461794,3310,20-03-2020 17:25,01-12-2020 00:00
3,id5416547,1743,17-03-2020 10:17,25-03-2020 11:00
4,id8238421,1859,01-03-2020 11:42,01-03-2020 11:43
3,id8641743,2752,20-03-2020 15:44,21-04-2020 15:44
2,id2185490,3210,16-03-2020 16:28,01-12-2020 00:00
6,id7642700,3020,15-03-2020 14:21,15-03-2020 23:42
2,id8741631,5677,19-03-2020 12:28,01-12-2020 00:00


In [168]:
sales1

,SUBS_ID,FILIAL_ID,PROD_ID,ACT_DTTM
0,id4651830,1,1954,20-03-2020 14:59
1,7646509,5,6431,19-03-2020 13:00
2,id7412683,4,3313,22-03-2020 17:25


In [169]:
sales2

,FILIAL_ID,SUBS_ID,PROD_ID,ACT_DTTM
0,3,id5416547,1743,17-03-2020 10:17
1,4,NaN,7421,12-03-2020 11:25
2,7,8362218,9879,05-03-2020 11:42
3,2,2185490,3210,16-03-2020 16:28


In [170]:
#колонки не везде в нормальном порядке, поменяем местами
columns = ['SUBS_ID','FILIAL_ID','PROD_ID','ACT_DTTM']

sales2 = sales2[columns]
sales3 = sales3[columns]

In [172]:
sales3

,SUBS_ID,FILIAL_ID,PROD_ID,ACT_DTTM
0,id5764122,3,1499,18-03-2020 15:44
1,id7642700,6,3020,15-03-2020 14:21
2,id1374509,2,5677,17-03-2020 11:48


In [173]:
#и теперь можно соединять их в один
df = pd.concat([sales1, sales2, sales3], ignore_index = True)

In [174]:
df

,SUBS_ID,FILIAL_ID,PROD_ID,ACT_DTTM
0,id4651830,1,1954,20-03-2020 14:59
1,7646509,5,6431,19-03-2020 13:00
2,id7412683,4,3313,22-03-2020 17:25
3,id5416547,3,1743,17-03-2020 10:17
4,NaN,4,7421,12-03-2020 11:25
5,8362218,7,9879,05-03-2020 11:42
6,2185490,2,3210,16-03-2020 16:28
7,id5764122,3,1499,18-03-2020 15:44
8,id7642700,6,3020,15-03-2020 14:21
9,id1374509,2,5677,17-03-2020 11:48


In [175]:
#сбросим дубликаты по SUBS_ID
df = df.dropna(subset=['SUBS_ID'])
df

,SUBS_ID,FILIAL_ID,PROD_ID,ACT_DTTM
0,id4651830,1,1954,20-03-2020 14:59
1,7646509,5,6431,19-03-2020 13:00
2,id7412683,4,3313,22-03-2020 17:25
3,id5416547,3,1743,17-03-2020 10:17
5,8362218,7,9879,05-03-2020 11:42
6,2185490,2,3210,16-03-2020 16:28
7,id5764122,3,1499,18-03-2020 15:44
8,id7642700,6,3020,15-03-2020 14:21
9,id1374509,2,5677,17-03-2020 11:48


In [176]:
# где пропущен id, добавим его в начало каждого значения в колонке
df['SUBS_ID'] = df['SUBS_ID'].apply(lambda x: 'id'+x if x[0].isdigit() else x)

/opt/tljh/user/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [177]:
df

,SUBS_ID,FILIAL_ID,PROD_ID,ACT_DTTM
0,id4651830,1,1954,20-03-2020 14:59
1,id7646509,5,6431,19-03-2020 13:00
2,id7412683,4,3313,22-03-2020 17:25
3,id5416547,3,1743,17-03-2020 10:17
5,id8362218,7,9879,05-03-2020 11:42
6,id2185490,2,3210,16-03-2020 16:28
7,id5764122,3,1499,18-03-2020 15:44
8,id7642700,6,3020,15-03-2020 14:21
9,id1374509,2,5677,17-03-2020 11:48


In [179]:
#присоединим друг к другу два датафрейма
df_final = df.merge(logs, how = 'inner', on = ['SUBS_ID','PROD_ID'])

In [180]:
df_final

,SUBS_ID,FILIAL_ID,PROD_ID,ACT_DTTM,START_DTTM,END_DTTM
0,id4651830,1,1954,20-03-2020 14:59,20-03-2020 14:59,01-12-2020 00:00
1,id7646509,5,6431,19-03-2020 13:00,19-03-2020 13:00,19-03-2020 13:03
2,id5416547,3,1743,17-03-2020 10:17,17-03-2020 10:17,25-03-2020 11:00
3,id2185490,2,3210,16-03-2020 16:28,16-03-2020 16:28,01-12-2020 00:00
4,id7642700,6,3020,15-03-2020 14:21,15-03-2020 14:21,15-03-2020 23:42


In [181]:
df_final.dtypes

SUBS_ID       object
FILIAL_ID      int64
PROD_ID        int64
ACT_DTTM      object
START_DTTM    object
END_DTTM      object
dtype: object

In [183]:
#переведем даты в нормальный формат
df_final['START_DTTM'] = pd.to_datetime(df_final.START_DTTM, format='%d-%m-%Y %H:%M')
df_final['END_DTTM'] = pd.to_datetime(df_final.END_DTTM, format='%d-%m-%Y %H:%M')

In [184]:
df_final

,SUBS_ID,FILIAL_ID,PROD_ID,ACT_DTTM,START_DTTM,END_DTTM
0,id4651830,1,1954,20-03-2020 14:59,2020-03-20 14:59:00,2020-12-01 00:00:00
1,id7646509,5,6431,19-03-2020 13:00,2020-03-19 13:00:00,2020-03-19 13:03:00
2,id5416547,3,1743,17-03-2020 10:17,2020-03-17 10:17:00,2020-03-25 11:00:00
3,id2185490,2,3210,16-03-2020 16:28,2020-03-16 16:28:00,2020-12-01 00:00:00
4,id7642700,6,3020,15-03-2020 14:21,2020-03-15 14:21:00,2020-03-15 23:42:00


In [185]:
#продажа не засчитывается, если отключение (END_DTTM) произошло меньше чем через 5 минут после подключения (START_DTTM)
df_final['difference'] = df_final.END_DTTM - df_final.START_DTTM

In [186]:
df_final

,SUBS_ID,FILIAL_ID,PROD_ID,ACT_DTTM,START_DTTM,END_DTTM,difference
0,id4651830,1,1954,20-03-2020 14:59,2020-03-20 14:59:00,2020-12-01 00:00:00,255 days 09:01:00
1,id7646509,5,6431,19-03-2020 13:00,2020-03-19 13:00:00,2020-03-19 13:03:00,0 days 00:03:00
2,id5416547,3,1743,17-03-2020 10:17,2020-03-17 10:17:00,2020-03-25 11:00:00,8 days 00:43:00
3,id2185490,2,3210,16-03-2020 16:28,2020-03-16 16:28:00,2020-12-01 00:00:00,259 days 07:32:00
4,id7642700,6,3020,15-03-2020 14:21,2020-03-15 14:21:00,2020-03-15 23:42:00,0 days 09:21:00


In [187]:
#а вот и наши незасчитанные продажи
df_final = df_final.query('difference > "5m"')

In [188]:
df_final

,SUBS_ID,FILIAL_ID,PROD_ID,ACT_DTTM,START_DTTM,END_DTTM,difference
0,id4651830,1,1954,20-03-2020 14:59,2020-03-20 14:59:00,2020-12-01 00:00:00,255 days 09:01:00
2,id5416547,3,1743,17-03-2020 10:17,2020-03-17 10:17:00,2020-03-25 11:00:00,8 days 00:43:00
3,id2185490,2,3210,16-03-2020 16:28,2020-03-16 16:28:00,2020-12-01 00:00:00,259 days 07:32:00
4,id7642700,6,3020,15-03-2020 14:21,2020-03-15 14:21:00,2020-03-15 23:42:00,0 days 09:21:00


In [189]:
#сохраним в список их SUBS_ID
idlist = df_final.SUBS_ID.tolist()

In [190]:
idlist 

['id4651830', 'id5416547', 'id2185490', 'id7642700']